<a href="https://colab.research.google.com/github/NabilahLianda/Kecerdasan-Buatan/blob/main/Kode_Program_KCB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# 1. Dataset
data = {
    'Nama': ['Sucianto', 'Riadi', 'Juliana', 'Vivin Rhmaiani', 'Suyono', 'Dinauli Manik',
             'Veny Setia Ningrum', 'Junedi Silalahi', 'Poltak Silaen', 'Median Sahara Manurung',
             'Edi Sopian Pohan', 'Herman', 'Suman', 'Kiki Kurniawan'],
    'Tidak Absen Masuk': [6, 10, 4, 7, 9, 11, 26, 16, 11, 12, 13, 21, 17, 16],
    'Sakit': [2, 10, 8, 6, 10, 4, 6, 3, 4, 7, 4, 11, 3, 1],
    'Izin': [15, 6, 11, 8, 7, 6, 4, 11, 2, 10, 6, 8, 10, 4],
    'Alpa': [0, 1, 0, 1, 5, 1, 2, 7, 11, 18, 12, 11, 9, 9],
    'Tidak Absen Pulang': [10, 9, 0, 2, 16, 15, 27, 26, 40, 38, 34, 55, 31, 42]
}
df = pd.DataFrame(data)

# 2. Centroid awal: Riadi (C1), Poltak (C2), Edi (C3)
centroids = {
    'C1': df.iloc[1, 1:].to_numpy(dtype=float),   # Riadi
    'C2': df.iloc[8, 1:].to_numpy(dtype=float),   # Poltak
    'C3': df.iloc[10, 1:].to_numpy(dtype=float),  # Edi
}

# 3. Fungsi Euclidean
def euclidean_distance(x, c):
    return np.sqrt(np.sum((x - c) ** 2))

# 4. Hitung jarak dan ranking
nama_list = []
jarak_list = []
ranking_list = []

for i in range(len(df)):
    row = df.iloc[i, 1:].to_numpy(dtype=float)
    nama = df.iloc[i, 0]
    d1 = euclidean_distance(row, centroids['C1'])
    d2 = euclidean_distance(row, centroids['C2'])
    d3 = euclidean_distance(row, centroids['C3'])

    distances = {'C1': d1, 'C2': d2, 'C3': d3}
    sorted_centroids = sorted(distances, key=distances.get)  # Urut dari jarak terpendek

    # Buat ranking: 2 = terdekat, 1 = sedang, 0 = terjauh
    ranking = {centroid: rank for rank, centroid in enumerate(reversed(sorted_centroids))}

    nama_list.append(nama)
    jarak_list.append({'Nama': nama, 'C1': round(d1, 5), 'C2': round(d2, 5), 'C3': round(d3, 5)})
    ranking_list.append({'Nama': nama, 'C1': ranking['C1'], 'C2': ranking['C2'], 'C3': ranking['C3']})

# 5. Buat DataFrame
jarak_df = pd.DataFrame(jarak_list)
ranking_df = pd.DataFrame(ranking_list)

# --- OUTPUT ---
pd.set_option("display.colheader_justify", "center")
pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", None)

# Format angka
float_format = "{:^10.5f}".format

# Formatter agar Nama rata kiri
jarak_formatters = {
    'Nama': lambda x: f'{x:<25}',  # Nama rata kiri, lebar 25
    'C1': float_format,
    'C2': float_format,
    'C3': float_format
}

ranking_formatters = {
    'Nama': lambda x: f'{x:<25}'  # Nama rata kiri juga di ranking
}

# 1. Dataset Pegawai
print("\nDataset Pegawai:")
print(df.to_string(index=False, justify='center', formatters=jarak_formatters))

# 2a. Jarak
print("\nJarak ke Centroid (Iterasi Pertama):")
print(jarak_df.to_string(index=False, justify='center', formatters=jarak_formatters))

# 2b. Ranking
print("\nPengelompokkan Jarak ke Centroid (0 = terjauh, 1= alternatif, 2 = terdekat):")
print(ranking_df.to_string(index=False, justify='center', formatters=ranking_formatters))



Dataset Pegawai:
           Nama            Tidak Absen Masuk  Sakit  Izin  Alpa  Tidak Absen Pulang
Sucianto                           6            2     15     0           10        
Riadi                             10           10      6     1            9        
Juliana                            4            8     11     0            0        
Vivin Rhmaiani                     7            6      8     1            2        
Suyono                             9           10      7     5           16        
Dinauli Manik                     11            4      6     1           15        
Veny Setia Ningrum                26            6      4     2           27        
Junedi Silalahi                   16            3     11     7           26        
Poltak Silaen                     11            4      2    11           40        
Median Sahara Manurung            12            7     10    18           38        
Edi Sopian Pohan                  13            4      6  

In [ ]:
while True:
    print(f"\n--- Iterasi ke-{iteration} ---")

    jarak_df = assign_cluster(df, centroids)
    cluster_new = jarak_df['Cluster'].tolist()

    # Cetak tabel jarak dulu
    print("\nJarak ke Centroid:")
    print(jarak_df[['Nama', 'C1', 'C2', 'C3']].to_string(index=False, justify='center', formatters=jarak_formatters))

    # Buat ranking jarak (0=terjauh, 1=alternatif, 2=terdekat)
    def rank_distances(row):
        distances = [row['C1'], row['C2'], row['C3']]
        sorted_dist = sorted(distances)
        ranks = [0, 0, 0]
        for i, d in enumerate(distances):
            # Ranking: 2 untuk terdekat, 0 terjauh
            ranks[i] = 2 - sorted_dist.index(d)
        return ranks

    ranking = jarak_df.apply(rank_distances, axis=1, result_type='expand')
    ranking.columns = ['C1', 'C2', 'C3']

    # Tambahkan kolom Nama agar tabel rapi
    ranking.insert(0, 'Nama', jarak_df['Nama'])

    print("\nPengelompokkan Jarak ke Centroid (0=terjauh,1=alternatif,2=terdekat):")
    print(ranking.to_string(index=False, justify='center', formatters=jarak_formatters))

    if cluster_old == cluster_new:
        print("\nCluster tidak berubah, iterasi dihentikan.")
        break
    else:
        cluster_old = cluster_new
        centroids = calculate_new_centroids(df, jarak_df)
        iteration += 1



--- Iterasi ke-2 ---

Jarak ke Centroid:
           Nama               C1         C2         C3    
Sucianto                   8.16837    39.95970   26.66908 
Riadi                      4.90465    38.61491   25.43698 
Juliana                    9.90230    48.82053   35.33893 
Vivin Rhmaiani             6.81094    45.67761   32.01000 
Suyono                     9.11348    31.38117   18.53753 
Dinauli Manik              8.08634    32.49786   19.37111 
Veny Setia Ningrum         26.27525   22.77669   13.40298 
Junedi Silalahi            20.43010   21.05812   6.69627  
Poltak Silaen              33.74990   8.15135    12.32234 
Median Sahara Manurung     34.01552   12.83658   12.20000 
Edi Sopian Pohan           28.23808   12.30628   5.76541  
Herman                     49.32601   12.48110   25.04077 
Suman                      25.62009   15.86751   2.49800  
Kiki Kurniawan             35.94517   5.86894    12.17539 

Pengelompokkan Jarak ke Centroid (0=terjauh,1=alternatif,2=terdekat):
  